In [ ]:
import os
import pandas as pd
import holidays
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from datetime import datetime

## Import data

In [ ]:
# Read the zones
zones = pd.read_csv("taxi_zone_lookup.csv")

# Grab Manhattan only
manhattan = zones[zones['Borough'] == 'Manhattan']

# See them
print(manhattan['Zone'].unique())

# Retain manhattan location IDs
manhattan_ids = manhattan['LocationID'].unique()

# See which borough you are looking for
borough = manhattan['Zone'].str.contains("Madison")

# Pluck it
results = manhattan[borough]
print(results)

# Grab location ids
location_id = results['LocationID'].values
location_id

['Alphabet City' 'Battery Park' 'Battery Park City' 'Bloomingdale'
 'Central Harlem' 'Central Harlem North' 'Central Park' 'Chinatown'
 'Clinton East' 'Clinton West' 'East Chelsea' 'East Harlem North'
 'East Harlem South' 'East Village' 'Financial District North'
 'Financial District South' 'Flatiron' 'Garment District'
 "Governor's Island/Ellis Island/Liberty Island" 'Gramercy'
 'Greenwich Village North' 'Greenwich Village South' 'Hamilton Heights'
 'Highbridge Park' 'Hudson Sq' 'Inwood' 'Inwood Hill Park' 'Kips Bay'
 'Lenox Hill East' 'Lenox Hill West' 'Lincoln Square East'
 'Lincoln Square West' 'Little Italy/NoLiTa' 'Lower East Side'
 'Manhattan Valley' 'Manhattanville' 'Marble Hill'
 'Meatpacking/West Village West' 'Midtown Center' 'Midtown East'
 'Midtown North' 'Midtown South' 'Morningside Heights' 'Murray Hill'
 'Penn Station/Madison Sq West' 'Randalls Island' 'Roosevelt Island'
 'Seaport' 'SoHo' 'Stuy Town/Peter Cooper Village'
 'Sutton Place/Turtle Bay North' 'Times Sq/Theatr

array([186])

## Preprocessing

In [144]:
data = pd.read_parquet('./raw_data/yellow_tripdata_2022-01.parquet')

In [145]:
# Let's keep pickup and dropoff, passenger count, distance, PU/DO location, tip_amount, total_amount (doesn't include cash tips but other tip yeah)
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [146]:
data.keys()

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [147]:
cols_to_keep = ['tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance',
       'PULocationID', 'DOLocationID', 'fare_amount', 
       'tip_amount', 'total_amount']

In [148]:
# Shrink data
data = data[cols_to_keep]
data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount
0,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,142,236,14.5,3.65,21.95
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,236,42,8.0,4.00,13.30
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,166,166,7.5,1.76,10.56
3,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,114,68,8.0,0.00,11.80
4,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,68,163,23.5,3.00,30.30


In [149]:
# Keep pickup ID in that location
if len(location_id) < 2:
    data = data[data["PULocationID"] == location_id[0]].drop(columns=["PULocationID"], axis=1)
else:
    data = data[data["PULocationID"].isin(location_id)].drop(columns=["PULocationID"], axis=1)

In [150]:
data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,DOLocationID,fare_amount,tip_amount,total_amount
105,2022-01-01 00:09:50,2022-01-01 00:13:40,5.0,0.56,68,4.5,0.00,8.30
187,2022-01-01 00:42:32,2022-01-01 01:02:35,1.0,4.29,263,16.5,4.06,24.36
197,2022-01-01 00:51:09,2022-01-01 01:03:16,2.0,1.51,114,9.5,3.33,16.63
720,2022-01-01 00:15:24,2022-01-01 00:50:41,1.0,2.20,48,20.5,0.00,24.30
930,2022-01-01 00:40:41,2022-01-01 00:46:50,1.0,0.59,234,5.5,0.00,9.30


In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80580 entries, 105 to 2463930
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tpep_pickup_datetime   80580 non-null  datetime64[us]
 1   tpep_dropoff_datetime  80580 non-null  datetime64[us]
 2   passenger_count        79775 non-null  float64       
 3   trip_distance          80580 non-null  float64       
 4   DOLocationID           80580 non-null  int64         
 5   fare_amount            80580 non-null  float64       
 6   tip_amount             80580 non-null  float64       
 7   total_amount           80580 non-null  float64       
dtypes: datetime64[us](2), float64(5), int64(1)
memory usage: 5.5 MB


In [152]:
# Option: keep dropoff ID in Mnahattan
data = data[data['DOLocationID'].isin(manhattan_ids)]

In [153]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75830 entries, 105 to 2463871
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tpep_pickup_datetime   75830 non-null  datetime64[us]
 1   tpep_dropoff_datetime  75830 non-null  datetime64[us]
 2   passenger_count        75108 non-null  float64       
 3   trip_distance          75830 non-null  float64       
 4   DOLocationID           75830 non-null  int64         
 5   fare_amount            75830 non-null  float64       
 6   tip_amount             75830 non-null  float64       
 7   total_amount           75830 non-null  float64       
dtypes: datetime64[us](2), float64(5), int64(1)
memory usage: 5.2 MB


In [154]:
# Force datetime
data["tpep_pickup_datetime"] = pd.to_datetime(data["tpep_pickup_datetime"])
data["tpep_dropoff_datetime"] = pd.to_datetime(data["tpep_dropoff_datetime"])

In [155]:
# Set index to pickups
data = data.set_index("tpep_pickup_datetime", drop=True).sort_index()
data.head()

,tpep_dropoff_datetime,passenger_count,trip_distance,DOLocationID,fare_amount,tip_amount,total_amount
tpep_pickup_datetime,,,,,,,
2021-12-31 23:24:49,2021-12-31 23:35:30,1.0,0.33,100,7.5,1.70,13.00
2022-01-01 00:02:44,2022-01-01 00:14:53,2.0,1.90,114,9.5,3.30,16.60
2022-01-01 00:03:38,2022-01-01 00:19:06,1.0,5.66,87,18.5,4.46,26.76
2022-01-01 00:09:50,2022-01-01 00:13:40,5.0,0.56,68,4.5,0.00,8.30
2022-01-01 00:13:10,2022-01-01 00:17:03,1.0,0.87,234,5.0,1.00,9.80


In [156]:
# Aggregation options
data_time = data.resample("30min").agg({
    'passenger_count': 'sum',
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'total_amount': 'sum'
})
data_time.head()

,passenger_count,trip_distance,fare_amount,tip_amount,total_amount
tpep_pickup_datetime,,,,,
2021-12-31 23:00:00,1.0,0.33,7.5,1.70,13.00
2021-12-31 23:30:00,0.0,0.00,0.0,0.00,0.00
2022-01-01 00:00:00,31.0,42.67,204.5,31.53,304.43
2022-01-01 00:30:00,24.0,30.09,173.0,34.42,268.22
2022-01-01 01:00:00,35.0,47.32,264.2,49.26,418.06


In [157]:
data.head()

,tpep_dropoff_datetime,passenger_count,trip_distance,DOLocationID,fare_amount,tip_amount,total_amount
tpep_pickup_datetime,,,,,,,
2021-12-31 23:24:49,2021-12-31 23:35:30,1.0,0.33,100,7.5,1.70,13.00
2022-01-01 00:02:44,2022-01-01 00:14:53,2.0,1.90,114,9.5,3.30,16.60
2022-01-01 00:03:38,2022-01-01 00:19:06,1.0,5.66,87,18.5,4.46,26.76
2022-01-01 00:09:50,2022-01-01 00:13:40,5.0,0.56,68,4.5,0.00,8.30
2022-01-01 00:13:10,2022-01-01 00:17:03,1.0,0.87,234,5.0,1.00,9.80


In [158]:
pickups = data.resample("30min").size()
data_time['pickup_count'] = pickups
data_time['weekday'] = data_time.index.weekday >= 5 # Doesn't include Friday
data_time.head()

,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,pickup_count,weekday
tpep_pickup_datetime,,,,,,,
2021-12-31 23:00:00,1.0,0.33,7.5,1.70,13.00,1,False
2021-12-31 23:30:00,0.0,0.00,0.0,0.00,0.00,0,False
2022-01-01 00:00:00,31.0,42.67,204.5,31.53,304.43,18,True
2022-01-01 00:30:00,24.0,30.09,173.0,34.42,268.22,16,True
2022-01-01 01:00:00,35.0,47.32,264.2,49.26,418.06,27,True


In [159]:
# Grab holiday
us_holidays = holidays.US(years=range(2022, 2025)) # 2022 to 2024
us_holidays.observed = False # remove weird observed (day falls on Sunday, so kick it to Monday)
for date, name in sorted(us_holidays.items()):
    print(date, name)

2022-01-01 New Year's Day
2022-01-17 Martin Luther King Jr. Day
2022-02-21 Washington's Birthday
2022-05-30 Memorial Day
2022-06-19 Juneteenth National Independence Day
2022-07-04 Independence Day
2022-09-05 Labor Day
2022-11-11 Veterans Day
2022-11-24 Thanksgiving
2022-12-25 Christmas Day
2023-01-01 New Year's Day
2023-01-16 Martin Luther King Jr. Day
2023-02-20 Washington's Birthday
2023-05-29 Memorial Day
2023-06-19 Juneteenth National Independence Day
2023-07-04 Independence Day
2023-09-04 Labor Day
2023-11-11 Veterans Day
2023-11-23 Thanksgiving
2023-12-25 Christmas Day
2024-01-01 New Year's Day
2024-01-15 Martin Luther King Jr. Day
2024-02-19 Washington's Birthday
2024-05-27 Memorial Day
2024-06-19 Juneteenth National Independence Day
2024-07-04 Independence Day
2024-09-02 Labor Day
2024-11-11 Veterans Day
2024-11-28 Thanksgiving
2024-12-25 Christmas Day


In [160]:
holidays_boi = [x[0] for x in us_holidays.items()]

In [161]:
data_time['is_holiday'] = [x.date() in holidays_boi for x in data_time.index]

In [162]:
data_time.head()

,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,pickup_count,weekday,is_holiday
tpep_pickup_datetime,,,,,,,,
2021-12-31 23:00:00,1.0,0.33,7.5,1.70,13.00,1,False,False
2021-12-31 23:30:00,0.0,0.00,0.0,0.00,0.00,0,False,False
2022-01-01 00:00:00,31.0,42.67,204.5,31.53,304.43,18,True,True
2022-01-01 00:30:00,24.0,30.09,173.0,34.42,268.22,16,True,True
2022-01-01 01:00:00,35.0,47.32,264.2,49.26,418.06,27,True,True
